In [1]:
!pip install --upgrade pip \
&& pip config set global.extra-index-url https://pip.repos.neuron.amazonaws.com \
&& pip install --upgrade optimum[neuronx] \
&& pip install --upgrade optimum-neuron neuronx-cc transformers_neuronx neuronx_distributed transformers torch-neuronx accelerate diffusers triton protobuf \
&& pip install faiss-cpu numpy scikit-learn pandas

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Writing to /root/.config/pip/pip.conf
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached optimum-1.24.0-py3-none-any.whl.metadata (21 kB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
INFO: pip is looking at multiple versions of optimum-neuron to determine which version is compatible with other requirements. This could take a while.
  Using cached optimum_neuron-0.0.27-py3-none-any.whl.metadata (15 kB)
  Using cached optimum_neuron-0.0.26-py3-none-any.whl.metadata (15 kB)
  Using cached optimum_neuron-0.0.25-py3-none-any.whl.metadata (15 kB)
  Using cached optimum_neuron-0.0.24-py3-none-any.whl.metadata (15 kB)
  Using cached transformers-4.41.1-py3-none-any.whl.metadata (43 kB)
  Using cached optimum_neuron-0.0.23-py3-none-any.whl.metadata (14 kB)
  Using cached optimum_neuron-0.0.22-py3-none-any.whl.metadata (14 kB)
  Using cached 

In [2]:
## https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews?resource=download
import pandas as pd

def load_amazon_books():
    url = "file:///home/ubuntu/Books_rating.csv"
    df = pd.read_csv(url,nrows=10000)
    #df = pd.read_csv(url)
    df = df[['Id', 'Title', 'User_id', 'review/helpfulness', 'review/score', 'review/time', 'review/summary', 'review/text']]
    return df
    
books_df = load_amazon_books()
books_df.head()

,Id,Title,User_id,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,A30TK6U7DNS82R,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,A3UH4UZ4RSVO82,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,A2MVUWT453QH61,7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,A22X4XUPKF66MR,3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [3]:
## Step 3: Deploy vLLM on Trainium (NxD) with Config YAML for main model
from vllm import LLM, SamplingParams
import yaml
import os

os.environ["NEURON_RT_VISIBLE_CORES"] = "0-1"

# model
model_vllm_config_yaml = """
model: "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
tensor_parallel_size: 2
max_num_seqs: 1
max_model_len: 512
override_neuron_config: {}
device: "neuron"
"""
model_vllm_config = yaml.safe_load(model_vllm_config_yaml)
llm_model = LLM(**model_vllm_config)

WARNING 03-11 01:24:40 _custom_ops.py:18] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")


/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


INFO 03-11 01:24:45 config.py:901] Defaulting to use mp for distributed inference
WARNING 03-11 01:24:45 config.py:376] Async output processing is only supported for CUDA or TPU. Disabling it for other platforms.
INFO 03-11 01:24:45 llm_engine.py:226] Initializing an LLM engine (v0.1.dev2830+g22c56ee) with config: model='deepseek-ai/DeepSeek-R1-Distill-Llama-8B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Llama-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=512, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cpu, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=Observabil

/opt/conda/lib/python3.10/site-packages/libneuronxla/neuron_cc_wrapper.py:196: SyntaxWarning: str format compiler_flags is discouraged as its handling involves repeated joining and splitting, which can easily make mistakes if something is quoted or escaped. Use list[str] instead. Refer to documentation of the Python subprocess module for details.
  warnings.warn(SyntaxWarning(


2025-03-11 01:25:07.000298:  92076  INFO ||NEURON_CC_WRAPPER||: Using a cached neff at /var/tmp/neuron-compile-cache/neuronxcc-2.16.372.0+4a9b2326/MODULE_0b0de4d2c192c0c83660+613edded/model.neff


/opt/conda/lib/python3.10/site-packages/libneuronxla/neuron_cc_wrapper.py:196: SyntaxWarning: str format compiler_flags is discouraged as its handling involves repeated joining and splitting, which can easily make mistakes if something is quoted or escaped. Use list[str] instead. Refer to documentation of the Python subprocess module for details.
  warnings.warn(SyntaxWarning(


2025-03-11 01:25:07.000354:  92077  INFO ||NEURON_CC_WRAPPER||: Using a cached neff at /var/tmp/neuron-compile-cache/neuronxcc-2.16.372.0+4a9b2326/MODULE_35c6e080f021e405f235+613edded/model.neff
2025-Mar-11 01:25:07.0763 91338:92060 [0] net_plugin.cc:70 CCOM WARN NET/Plugin: Error: libnccom-net.so load failed. libfabric.so.1: cannot open shared object file: No such file or directory. Please make sure to install the latest version of libfabric from https://efa-installer.amazonaws.com/aws-efa-installer-latest.tar.gz.


In [4]:
## Step 4: Generate Expanded Interests Using LLM Reasoning
def expand_interest(user_reviews):
    prompt = f"Given the following book interests, suggest related book themes, specific genres, and example book titles: {user_reviews}"
    output = llm_model.generate([prompt])[0]  # Extract first response object
    expanded_text = output.outputs[0].text if hasattr(output, "outputs") and output.outputs else str(output)
    return expanded_text.strip()

# Example user with past book reviews
user_reviews = "I love epic sci-fi novels with deep world-building."
expanded_interest = expand_interest(user_reviews)
print("Expanded Interest:", expanded_interest)

Processed prompts: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.51it/s, est. speed input: 46.93 toks/s, output: 24.22 toks/s]

Expanded Interest: I’m drawn to storylines where characters deal with existential questions and explore the themes


In [5]:
## Step 5: trace Bert for generating embeddings 
import torch
import torch_neuronx
from transformers import AutoModel, AutoTokenizer

model_name = "bert-base-uncased"
export_dir = "bert_feature_dir"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModel.from_pretrained(model_name)
model.eval()  

# Create dummy inputs for tracing
batch_size = 1
sequence_length = 512
dummy_inputs = (
    torch.ones((batch_size, sequence_length), dtype=torch.long),  # input_ids
    torch.ones((batch_size, sequence_length), dtype=torch.long)   # attention_mask
)

# Trace and compile the model for AWS Neuron
traced_model = torch_neuronx.trace(model, dummy_inputs)
traced_model.save(export_dir + "/model.pt")

print(f"Model successfully exported to {export_dir}")

..
Compiler status PASS
Model successfully exported to bert_feature_dir


In [6]:
from optimum.neuron import NeuronModelForFeatureExtraction
from transformers import AutoTokenizer
import torch
import numpy as np

#import os
#os.environ["NEURON_RT_VISIBLE_CORES"] = "2-3"

# Load compiled model (assuming you used "bert_feature_dir")
model_dir = "bert_feature_dir"
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
embedding_model = NeuronModelForFeatureExtraction.from_pretrained(model_dir)

/opt/conda/lib/python3.10/site-packages/neuronx_distributed/modules/moe/expert_mlps.py:11: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from neuronx_distributed.modules.moe.blockwise import (


In [7]:
import warnings
from transformers import logging

logging.set_verbosity_error()
warnings.simplefilter("ignore")

def get_embedding(text):
    """Compute BERT embeddings for a given text."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val for key, val in inputs.items()}
    with torch.no_grad():
        outputs = embedding_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

sample_text = books_df["review/text"].iloc[0]  # Get first review
embedding = get_embedding(sample_text)
print("Sample Embedding Shape:", embedding.shape)
print("Embedding Sample:", embedding[:10])  # Print first 10 values

Sample Embedding Shape: (768,)
Embedding Sample: [ 0.01468755 -0.1420664   0.31523687 -0.031531    0.23696186 -0.09850309
 -0.02450292  0.58309984 -0.08324284  0.02135697]


In [9]:
books_df["embedding"] = books_df["review/text"].apply(lambda x: get_embedding(x).tolist())

In [10]:
print(books_df.head())
print(books_df["embedding"].iloc[0][:10])  # Print first 10 values of the first embedding

           Id                           Title         User_id  \
0  1882931173  Its Only Art If Its Well Hung!   AVCGYZL8FQQTD   
1  0826414346        Dr. Seuss: American Icon  A30TK6U7DNS82R   
2  0826414346        Dr. Seuss: American Icon  A3UH4UZ4RSVO82   
3  0826414346        Dr. Seuss: American Icon  A2MVUWT453QH61   
4  0826414346        Dr. Seuss: American Icon  A22X4XUPKF66MR   

  review/helpfulness  review/score  review/time  \
0                7/7           4.0    940636800   
1              10/10           5.0   1095724800   
2              10/11           5.0   1078790400   
3                7/7           4.0   1090713600   
4                3/3           4.0   1107993600   

                                    review/summary  \
0           Nice collection of Julie Strain images   
1                                Really Enjoyed It   
2  Essential for every personal and Public Library   
3  Phlip Nel gives silly Seuss a serious treatment   
4                           Good

In [13]:
## Step 6: Store and Retrieve Embeddings Using FAISS
import faiss

# Convert embeddings into numpy array
embeddings_matrix = np.array(books_df["embedding"].tolist()).astype("float32")

# Create FAISS index
index = faiss.IndexFlatL2(768)  # 768 = embedding dimension from BERT
index.add(embeddings_matrix)

# Save index for reuse
faiss.write_index(index, "books_faiss_index.faiss")
print("FAISS index saved.")

FAISS index saved.


In [14]:
## Step 7: Implement book recommendations 
def recommend_books(user_reviews, top_k=5):
    # Generate expanded interest text
    expanded_interest = expand_interest(user_reviews)  
    
    # Generate embedding for expanded interest
    query_embedding = get_embedding(expanded_interest).astype("float32").reshape(1, -1)
    
    # Search in FAISS index
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve books from dataset
    recommended_books = books_df.iloc[indices[0]]  # Fetch rows by index

    return recommended_books


# Example: Find books similar to an interest
user_reviews = "I love epic sci-fi novels with deep world-building."
recommended_books = recommend_books(user_reviews)

print("Expanded Interest:", expand_interest(user_reviews))
print("Recommended Books:")
print(recommended_books[["Title", "review/summary"]])


Processed prompts: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.53it/s, est. speed input: 47.66 toks/s, output: 24.60 toks/s]

Expanded Interest: Every time I try to visualize a book's setting, I imagine it as a
Recommended Books:
                              Title  \
8117                     Foundation   
9664                    Pushing Ice   
2995  Space Stations and Graveyards   
3600                         Island   
3608                         Island   

                                      review/summary  
8117                      The Softest of Soft Sci Fi  
9664                             Absolutely AWESOME!  
2995  Good combination of science fiction and horror  
3600                      Raises Some Good Questions  
3608                      Raises Some Good Questions  


In [15]:
## Now that we checked how single expended interest works, lets precompute the embeddings for the entire dataset before 
## querying will significantly speed up the recommendation process and make the system more efficient. 
## Right now, recommend_books(user_reviews) computes everything on the fly, which is inefficient.
import numpy as np

# Compute embeddings for all book reviews in advance
if "embedding" not in books_df.columns:
    books_df["embedding"] = books_df["review/text"].apply(lambda x: get_embedding(x).tolist())

# Convert to NumPy array (FAISS requires float32)
book_embeddings = np.array(books_df["embedding"].tolist()).astype("float32")

# Normalize embeddings for better FAISS search (optional but recommended)
import faiss
faiss.normalize_L2(book_embeddings)

# Build FAISS index
index = faiss.IndexFlatL2(book_embeddings.shape[1])  # L2 search index
index.add(book_embeddings)  # Add precomputed embeddings to FAISS


In [16]:
# Example user with past book reviews
user_reviews = "I love epic sci-fi novels with deep world-building."
recommended_books = recommend_books(user_reviews)

print("Expanded Interest:", expand_interest(user_reviews))
print("Recommended Books:")
print(recommended_books[["Title", "review/summary"]])


Processed prompts: 100%|██████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.53it/s, est. speed input: 47.59 toks/s, output: 24.56 toks/s]

Expanded Interest: I enjoy speculative fiction that feels realistic, where the story seamlessly integrates into the real
Recommended Books:
                                                  Title  \
434                  Night World: Daughters Of Darkness   
6538  Tess of the D'Urbervilles: A pure woman (Harpe...   
5740  Prodigal Son (Dean Koontz's Frankenstein, Book 1)   
7912                                         Foundation   
5811                          Night Visitor (Timeswept)   

                          review/summary  
434                  It's got me hooked.  
6538             Tess ofthe d'Ubervilles  
5740                            Gripping  
7912       The greatest sci-fi book ever  
5811  original and lush with description  
